# importing neccessery library

In [28]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

# 1.reading dataset

In [23]:
data = pd.read_csv('IRIS.csv')

In [6]:
data.head()

,sepal-length,sepal-width,petal-length,petal-width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# 2.find each feature data type

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal-length  150 non-null    float64
 1   sepal-width   150 non-null    float64
 2   petal-length  150 non-null    float64
 3   petal-width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## **from the above output we can understand the 'species' column needs a category encoding and as the dataset doesn't have a lot of feature it's be good to use one-hot encoder**

# 3.finding data statics:

In [9]:
data.describe()

,sepal-length,sepal-width,petal-length,petal-width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


# 4.checking for class imbalance and missing value and outliers

In [11]:
data['species'].value_counts()

species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [13]:
data.isnull().sum()

sepal-length    0
sepal-width     0
petal-length    0
petal-width     0
species         0
dtype: int64

## checking for outliers using z-score

In [20]:
#calculate z-score for each column of data
z_scores = np.abs((data - data.mean())/data.std(ddof=0))
#define a threshold z-score value beyond which data point is considered an outlier
threshold = 3.5
#Identify outliers in each column
outliers = data[(z_scores>threshold).any(axis=1)]
#printing outliers
print(outliers)

Empty DataFrame
Columns: [sepal-length, sepal-width, petal-length, petal-width]
Index: []


there was no outlier in the dataset :)

# spliting the dataset

In [27]:
X = data.drop('species',axis=1)
y = data['species']
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y)

categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

# preprocessing and category encoding with piplines
in the following we are going to scale the numerical data and encode the 'species' column using one hot encoding using pipelines.
## how to choose:
based on the information from step 2 and 3 we conclude the best scaler for this dataset is standard scaler and as the dataset doesn't have a lot of features the one hot encoder are the best choises.

In [30]:
#define a pipline for scaling
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
#define a pipline for encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
#defining the logistic regression model
model = LogisticRegression()
#using pipline to run the model
pipeline = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
    ])

In [31]:
#finding the model accuracy
pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_valid)
score = accuracy_score(y_valid, pred)
print('accurancy:', score)

accurancy: 0.9210526315789473
